In [3]:
%%capture
# install PyTorch Geometric if running on Google Colab
import sys
if 'google.colab' in sys.modules:
  !pip install node2vec
  import torch

  def format_pytorch_version(version):
    return version.split('+')[0]

  TORCH_version = torch.__version__
  TORCH = format_pytorch_version(TORCH_version)

  def format_cuda_version(version):
    return 'cu' + version.replace('.', '')

  CUDA_version = torch.version.cuda
  CUDA = format_cuda_version(CUDA_version)

  !pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
  !pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
  !pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
  !pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
  !pip install torch-geometric

## Graph embedding-based methods

Such techniques seek to apply graph embedding techniques to obtain node-level or graph-level representations and further use the representations for similarity learning. For example, DeepWalk and Node2Vec can be used to extract meaningful embedding that can then be used to define a similarity function or to predict similarity scores. For example, in Tixier et al. (2015), node2vec was used for encoding node embeddings for representing a graph as an image. Specifically, two-dimensional (2D) histograms obtained from those node embeddings were passed to a classical 2D convolutional neural network (CNN) architecture designed for images. Such a simple yet powerful approach enabled good results to be obtained for many benchmark datasets.

In [4]:
# Method 1: Graph Embedding-based (Node2Vec)
import networkx as nx
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#from sklearn.preprocessing import LabelEncoder

# Load necessary libraries for each method
from node2vec import Node2Vec
import torch
import torch.nn as nn
import torch.optim as optim

# Create toy dataset with simple graphs
num_graphs = 10
graphs = [nx.erdos_renyi_graph(10, np.random.rand()) for _ in range(num_graphs)]

# Labels for graph similarity task (could be used for graph classification or regression)
labels = [np.random.choice([0,1]) for _ in range(num_graphs)]

# Function to generate 2D histogram from node embeddings
def generate_2d_histogram(node_embeddings, bins=16):
    # Flatten embeddings to create histograms
    embeddings = np.vstack(node_embeddings)
    histogram, xedges, yedges = np.histogram2d(embeddings[:, 0], embeddings[:, 1], bins=bins)
    return histogram

# Prepare graph-level 2D histograms from node embeddings
graph_histograms = []
for i, graph in enumerate(graphs):
    node2vec = Node2Vec(graph, dimensions=64, walk_length=10, num_walks=80, workers=4)
    model = node2vec.fit()
    node_embeddings = [model.wv.get_vector(str(node)) for node in graph.nodes()]
    histogram = generate_2d_histogram(node_embeddings)
    graph_histograms.append(histogram)

# Convert histograms into tensors for CNN
graph_histograms = np.array(graph_histograms)
graph_histograms = torch.tensor(graph_histograms, dtype=torch.float32)

# Split histograms into training and testing sets
train_histograms, test_histograms, train_labels, test_labels = train_test_split(graph_histograms, labels, test_size=0.5, random_state=42)

# Define a simple 2D CNN model for graph classification
class GraphCNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(GraphCNN, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(5, -1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize and train the CNN model
num_classes = 2  # Binary classification
input_channels = 1  # Single channel for histogram
bins = 16  # Same as used in generate_2d_histogram function

train_histograms = train_histograms.unsqueeze(1)  # Add channel dimension
test_histograms = test_histograms.unsqueeze(1)

cnn_model = GraphCNN(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

# Training loop
cnn_model.train()
epochs = 20
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = cnn_model(train_histograms)
    loss = criterion(outputs, torch.tensor(train_labels, dtype=torch.long))
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# Evaluate on test set
cnn_model.eval()
with torch.no_grad():
    test_outputs = cnn_model(test_histograms)
    predictions = torch.argmax(test_outputs, axis=1)
    accuracy = accuracy_score(test_labels, predictions.numpy())
    print(f"Test Accuracy: {accuracy}")

Computing transition probabilities:   0%|          | 0/10 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/10 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/10 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/10 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/10 [00:00<?, ?it/s]

...

Computing transition probabilities:   0%|          | 0/10 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/10 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/10 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/20, Loss: 0.6815312504768372
Epoch 2/20, Loss: 0.5630680322647095
Epoch 3/20, Loss: 0.41003280878067017
Epoch 4/20, Loss: 0.32633399963378906
Epoch 5/20, Loss: 0.22436682879924774
Epoch 6/20, Loss: 0.09085053205490112
Epoch 7/20, Loss: 0.04055032134056091
Epoch 8/20, Loss: 0.013849971815943718
Epoch 9/20, Loss: 0.008305830880999565
Epoch 10/20, Loss: 0.003659198060631752
Epoch 11/20, Loss: 0.0010380035964772105
Epoch 12/20, Loss: 0.00029954835190437734
Epoch 13/20, Loss: 0.00010289005876984447
Epoch 14/20, Loss: 8.603346213931218e-05
Epoch 15/20, Loss: 1.873927794804331e-05
Epoch 16/20, Loss: 3.4570541629364016e-06
Epoch 17/20, Loss: 1.0967236221404164e-06
Epoch 18/20, Loss: 1.7404512391294702e-06
Epoch 19/20, Loss: 5.412063728726935e-06
Epoch 20/20, Loss: 7.152556946721234e-08
Test Accuracy: 0.6


## Graph kernel-based methods

Graph kernel-based methods have generated a lot of interest in terms of capturing the similarity between graphs. These approaches compute the similarity between two graphs as a function of the similarities between some of their substructures. Different graph kernels exist based on the substructures they use, which include random walks, shortest paths, and subgraphs. As an example, a method called Deep Graph Kernels (DGK) (Yanardag et al., 2015) decomposes graphs into substructures that are viewed as "words". Then, natural language processing (NLP) approaches such as continuous bag of words (CBOW) and skip-gram are used to learn latent representations of the substructures. This way, the kernel between two graphs is defined based on the similarity of the substructure space.

In [5]:
import networkx as nx
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics.pairwise import cosine_similarity

# Create toy dataset with simple graphs
num_graphs = 10
graphs = [nx.erdos_renyi_graph(50, np.random.rand()) for _ in range(num_graphs)]

# Labels for graph similarity task (could be used for graph classification or regression)
labels = [np.random.choice([0,1]) for _ in range(num_graphs)]

# Decompose graphs into substructures using random walks
def random_walks_as_words(graph, walk_length=6, num_walks=10):
    """Generates 'words' from random walks on the graph."""
    walks = []
    for _ in range(num_walks):
        for node in graph.nodes():
            walk = [str(node)]
            for _ in range(walk_length - 1):
                neighbors = list(graph.neighbors(int(walk[-1])))
                if neighbors:
                    walk.append(str(np.random.choice(neighbors)))
                else:
                    break
            walks.append(" ".join(walk))
    return walks

# Create a "document" for each graph using its random walks
graph_documents = []
for graph in graphs:
    walks = random_walks_as_words(graph)
    graph_documents.append(" ".join(walks))  # Combine all walks into a single document

# Use CountVectorizer to generate a "bag of words" representation for each graph
vectorizer = CountVectorizer()
graph_bow = vectorizer.fit_transform(graph_documents)  # Sparse matrix of shape (num_graphs, num_features)

# Compute pairwise similarities between graphs using cosine similarity
graph_similarity_matrix = cosine_similarity(graph_bow)

# Use the similarity matrix as input features for a classification task
# Split dataset into training and testing sets
train_indices, test_indices = train_test_split(range(len(graphs)), test_size=0.5, random_state=42)

train_similarity = graph_similarity_matrix[np.ix_(train_indices, train_indices)]
test_similarity = graph_similarity_matrix[np.ix_(test_indices, train_indices)]  # Test against training similarities

train_labels = [labels[i] for i in train_indices]
test_labels = [labels[i] for i in test_indices]

# Train a classifier using the training similarity matrix
svm = SVC(kernel="precomputed")  # Precomputed kernel
svm.fit(train_similarity, train_labels)

# Predict on the test set
test_predictions = svm.predict(test_similarity)

# Evaluate the classification accuracy
accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy using Deep Graph Kernels (DGK): {accuracy}")

Test Accuracy using Deep Graph Kernels (DGK): 0.6


## GNN-based methods

With the emergence of deep learning (DL) techniques, GNNs have become a powerful new tool for learning representations on graphs. Such powerful models can be easily adapted to various tasks, including graph similarity learning. Furthermore, they present a key advantage with respect to other traditional graph embedding approaches. Indeed, while the latter generally learn the representation in an isolated stage, in this kind of approach, the representation learning and the target learning task are conducted jointly. Therefore, the GNN deep models can better leverage the graph features for the specific learning task. We have already seen an example of similarity learning using GNNs in Chapter 4, Unsupervised Graph Learning, where a two-branch network was trained to estimate the proximity distance between two graphs.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import networkx as nx
import numpy as np

# Create toy dataset with simple graphs
num_graphs = 10
graphs = [nx.erdos_renyi_graph(10, np.random.rand()) for _ in range(num_graphs)]

# Generate similarity labels (e.g., based on structural similarity)
labels = [np.random.rand() for _ in range(num_graphs)]  # Similarity score between 0 and 1

# Convert graphs to PyTorch Geometric Data objects
def nx_to_data(graph, label):
    edge_index = torch.tensor(list(graph.edges), dtype=torch.long).t().contiguous()
    x = torch.eye(graph.number_of_nodes(), dtype=torch.float)  # Node features as identity matrix
    return Data(x=x, edge_index=edge_index, y=torch.tensor([label], dtype=torch.float))

data_list = [nx_to_data(graph, labels[i]) for i, graph in enumerate(graphs)]

# Split dataset into training and testing sets
train_data, test_data = train_test_split(data_list, test_size=0.5, random_state=42)

# Create DataLoaders for batching
train_loader = DataLoader(train_data, batch_size=2, shuffle=True)
test_loader = DataLoader(test_data, batch_size=2, shuffle=False)

# Define a GNN model for graph-level representation
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x, edge_index, batch):
        x = self.relu(self.conv1(x, edge_index))
        x = self.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)  # Graph-level pooling
        x = self.fc(x)
        return x

# Define a two-branch similarity learning network
class SimilarityNet(nn.Module):
    def __init__(self, gnn, embedding_dim):
        super(SimilarityNet, self).__init__()
        self.gnn = gnn
        self.fc = nn.Linear(2 * embedding_dim, 1)  # Combine embeddings from two branches

    def forward(self, data1, data2):
        # Extract graph-level embeddings for both graphs
        x1 = self.gnn(data1.x, data1.edge_index, data1.batch)
        x2 = self.gnn(data2.x, data2.edge_index, data2.batch)
        # Concatenate embeddings and pass through a fully connected layer
        x = torch.cat([x1, x2], dim=1)
        similarity_score = self.fc(x)
        return similarity_score

# Initialize models and hyperparameters
input_dim = 10  # Number of node features
hidden_dim = 32
embedding_dim = 16
gnn = GNN(input_dim, hidden_dim, embedding_dim)
model = SimilarityNet(gnn, embedding_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
model.train()
epochs = 20
for epoch in range(epochs):
    epoch_loss = 0
    for data1, data2 in zip(train_loader, train_loader):  # Pair graphs for similarity learning
        optimizer.zero_grad()
        pred = model(data1, data2)
        target = (data1.y + data2.y) / 2  # Example: Average similarity score
        loss = criterion(pred.view(-1), target.view(-1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss / len(train_loader)}")

# Evaluate the model
model.eval()
predictions, targets = [], []
with torch.no_grad():
    for data1, data2 in zip(test_loader, test_loader):
        pred = model(data1, data2)
        target = (data1.y + data2.y) / 2
        predictions.extend(pred.view(-1).tolist())
        targets.extend(target.view(-1).tolist())

mse = mean_squared_error(targets, predictions)
print(f"Test Mean Squared Error: {mse}")